# ResNet Ablation Study

### 패키지 설처하기

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

### GPU 장착 확인하기

In [2]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### cats_vs_dogs 데이터셋 불러오기

In [3]:
# 새 링크로 URL 수정
setattr(tfds.image_classification.cats_vs_dogs, '_URL',
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

# 배치 크기 설정
BATCH_SIZE = 256

# 이미지 리사이즈와 정규화 전처리 함수 (224x224로 리사이즈)
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # 224x224로 리사이즈
    image = tf.cast(image, tf.float32) / 255.0  # [0, 1] 범위로 정규화
    return image, label

In [4]:
# 데이터셋 로드
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],  # 학습 및 검증용 데이터 분리
    as_supervised=True,
    shuffle_files=True,  # 파일을 셔플
    with_info=True,
)

# 전처리 적용 및 배치 크기 설정
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

### feature 정보 확인하기

In [5]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요.
print(ds_info.features)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})


### 데이터 개수 확인하기

In [6]:
# 데이터의 개수도 확인해 봅시다.
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(73, shape=(), dtype=int64)
tf.Tensor(19, shape=(), dtype=int64)


In [7]:
# 배치 처리 전 데이터 개수 확인
#ds_train_unbatched = ds_train.unbatch()
#ds_test_unbatched = ds_test.unbatch()

#train_size = sum(1 for _ in ds_train_unbatched)
#test_size = sum(1 for _ in ds_test_unbatched)

#print(f"학습 데이터셋 샘플 개수: {train_size}")
#print(f"검증 데이터셋 샘플 개수: {test_size}")

### 정규화 코드 작성하기

In [8]:
# Q. 이미지의 표현이 0과 1 사이로 들어오도록 직접 정규화 코드를 작성해봅시다.
def normalize_and_resize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

In [9]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img,
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

### 클래스 구성요소 확인하기

In [10]:
ds_info.features["label"].num_classes

2

In [11]:
ds_info.features["label"].names

['cat', 'dog']

# 블록 구성하기

## ResNet 기본 블록 만들기
- 34와 50은 모두 conv block 이라고 불리는 블럭 구조를 각각 3,4,6,3개씩 반복해서 쌓은 형태의 공통점을 가지고 있다
- 34는 block은 3x3 kernel인 convolution layer로만 구성되어 있다
- 50은 1x1 convolution이 앞뒤로 붙어 더 많은 레이어를 한 블록 내에 가지고 있다

## ResNet, PlainNet 차이
- ResNet은 skip connection과 residual network가 있다
- PlainNet은 skip connection이 없다

In [12]:
# ResNet 블록을 구성하는 함수 민들기
def build_resnet_block(x, filters, blocks, stride=1, is_plain=False, is_50=False):

    expansion = 4 if is_50 else 1
    # ResNet-50에서는 필터 확장을 위해 4를 사용, ResNet-34에서는 확장하지 않음

    for i in range(blocks): # 지정된 블록 수만큼 반복해서 Residual/Bottleneck Block을 쌓음
        stride_current = stride if i == 0 else 1 # 다운샘플링
        shortcut = x

        if is_50: # resnet-50 블록 구성
            
            # 1x1 합성곱으로 채널 수를 축소
            x = keras.layers.Conv2D(filters, kernel_size=1, strides=stride_current, padding='same', kernel_initializer='he_normal')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

            # 3x3 합성곱으로 공간 정보를 학습
            x = keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

            # 1x1 합성곱으로 채널 수를 확장
            x = keras.layers.Conv2D(filters * expansion, kernel_size=1, strides=1, padding='same', kernel_initializer='he_normal')(x)
            x = keras.layers.BatchNormalization()(x)
        
        else: # resnet-34 블록 구성
            
            # 3x3 합성곱으로 공간 정보를 학습
            x = keras.layers.Conv2D(filters, kernel_size=3, strides=stride_current, padding='same', kernel_initializer='he_normal')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

            # 다시 3x3 합성곱 적용
            x = keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
            x = keras.layers.BatchNormalization()(x)

        if not is_plain: # Residual 연결(resnet 34, 50에만 사용한다)
            if stride_current != 1 or keras.backend.int_shape(shortcut)[-1] != filters * expansion:
                shortcut = keras.layers.Conv2D(filters * expansion, kernel_size=1, strides=stride_current, padding='same', kernel_initializer='he_normal')(shortcut)
                shortcut = keras.layers.BatchNormalization()(shortcut)
            x = keras.layers.Add()([x, shortcut])

        x = keras.layers.Activation('relu')(x)

    return x

In [13]:
def build_resnet(input_shape, num_classes, is_50=False, is_plain=False):

    inputs = keras.layers.Input(shape=input_shape)

    # 초기 레이어
    x = keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal',name='input_layer')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    if is_50: # ResNet-50 블록 수
        layers = [3, 4, 6, 3]
        filters = [64, 128, 256, 512]
    else: # ResNet-34 블록 수
        layers = [3, 4, 6, 3]
        filters = [64, 128, 256, 512]

    # 각 스테이지별로 블록을 쌓습니다.
    for i in range(len(layers)):
        stride = 1 if i == 0 else 2
        x = build_resnet_block(x, filters=filters[i], blocks=layers[i], stride=stride, is_plain=is_plain, is_50=is_50)

    # 최종 레이어
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(num_classes, activation='softmax')(x)

    # 모델 이름 설정
    model_depth = 50 if is_50 else 34
    model_type = 'plainnet' if is_plain else 'resnet'
    model_name = f"{model_type}_{model_depth}"

    model = keras.models.Model(inputs=inputs, outputs=x, name=model_name)

    return model

In [14]:
def build_plainnet(input_shape, num_classes, is_50=False):

    return build_resnet(input_shape, num_classes, is_50=is_50, is_plain=True)

### ResNet-34

In [15]:
# 모델 빌드
resnet_34 = build_resnet(input_shape=(224, 224, 3), num_classes=2, is_50=False, is_plain=False)
resnet_34.summary()

Model: "resnet_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_layer (Conv2D)            (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         input_layer[0][0]                
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
__________________________________________________________________________________________

In [16]:
# resnet_34 = build_resnet(input_shape=(32, 32,3), is_50=False)
# resnet_34.summary()

### ResNet-50

In [17]:
# 모델 빌드
resnet_50 = build_resnet(input_shape=(224, 224, 3), num_classes=2, is_50=False, is_plain=False)
resnet_50.summary()

Model: "resnet_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_layer (Conv2D)            (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 112, 112, 64) 256         input_layer[0][0]                
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 112, 112, 64) 0           batch_normalization_36[0][0]     
__________________________________________________________________________________________

In [18]:
# resnet_50 = build_resnet(input_shape=(32, 32,3), is_50=True)
# resnet_50.summary()

### epoch 설정하기

In [19]:
EPOCH = 15

### resnet 34 학습하기

In [20]:
resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_34 = resnet_34.fit(
    ds_train,
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/15


/opt/conda/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


20/73 [=======>......................] - ETA: 1:07 - loss: 0.7566 - accuracy: 0.5641

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


24/73 [========>.....................] - ETA: 1:02 - loss: 0.7413 - accuracy: 0.5708

...

68/73 [==========================>...] - ETA: 6s - loss: 0.0030 - accuracy: 0.9999

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


73/73 [==============================] - ETA: 0s - loss: 0.0030 - accuracy: 0.9999

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


73/73 [==============================] - 104s 1s/step - loss: 0.0030 - accuracy: 0.9999 - val_loss: 1.1280 - val_accuracy: 0.7182


### resnet 50 학습하기

In [ ]:
resnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_50 = resnet_50.fit(
    ds_train,
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/15
21/73 [=======>......................] - ETA: 1:08 - loss: 0.7037 - accuracy: 0.5616

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


24/73 [========>.....................] - ETA: 1:05 - loss: 0.7001 - accuracy: 0.5643

26/73 [=========>....................] - ETA: 1:02 - loss: 0.6978 - accuracy: 0.5675

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


...

26/73 [=========>....................] - ETA: 1:02 - loss: 0.6302 - accuracy: 0.6469

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


35/73 [=============>................] - ETA: 50s - loss: 0.6299 - accuracy: 0.6473

Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


54/73 [=====================>........] - ETA: 25s - loss: 0.6226 - accuracy: 0.6546

## Plainnet 기본 블록 만들기
- 7x7 커널의 cnn 레이어와 스트라이드2를 사용한 maxpooling 레이어로 시작한다

In [ ]:
# PlainNet-34 모델
plainnet_34 = build_plainnet(input_shape=(224, 224, 3), num_classes=2, is_50=False)
plainnet_34.summary()

In [ ]:
# plainnet_34 = build_plainnet(input_shape=(32, 32,3), num_layers=34)
# plainnet_34.summary()

In [ ]:
# PlainNet-50 모델
plainnet_50 = build_plainnet(input_shape=(224, 224, 3), num_classes=2, is_50=False)
plainnet_50.summary()

In [ ]:
# plainnet_50 = build_plainnet(input_shape=(32, 32,3), num_layers=50)
# plainnet_50.summary()

In [ ]:
plainnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_plain_34 = plainnet_34.fit(
    ds_train,
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plainnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_plain_50 = plainnet_50.fit(
    ds_train,
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

### resnet_34과 plainnet_34 비교하기

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_34.history['loss'], 'r')
plt.plot(history_plain_34.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'plainnet_34'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_34.history['val_accuracy'], 'r')
plt.plot(history_plain_34.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'plainnet_34'], loc='upper left')
plt.show()

### resnet_50과 plainnet_50 비교하기

In [ ]:
plt.plot(history_50.history['loss'], 'r')
plt.plot(history_plain_50.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet_50', 'plainnet_50'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_50.history['val_accuracy'], 'r')
plt.plot(history_plain_50.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet_50', 'plainnet_50'], loc='upper left')
plt.show()

## 회고
- 이미지셋을 CIFAR-10으로 학습해버려서 cats_vs_dogs로 다시 학습했다
- cats_vs_dogs의 이미지셋은 url로 다운받았다
- (32,32,3)은 한 모델 학습하는데 5분 정도 였다면 (224, 224, 3)은 그에 비해 아주 오래 걸렸다
- loss는 큰 차이라고 생각하지 않았는데 acc가 큰 차이를 보였다